In [1]:
import torch
from torch import nn

torch.__version__

'2.0.0+cu117'

In [2]:
from torchvision import transforms

batch_size = 1
img_size = 224

train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.Resize(size=(64,64)),
    transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

test_transform = transforms.Compose([
    # transforms.Resize(230),
    transforms.Resize(size=(64,64)),
    # transforms.CenterCrop(img_size),
    # transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    transforms.ToTensor()
    ])

In [3]:
from torchvision import datasets
from torch.utils.data import DataLoader

train_set = datasets.ImageFolder("./dataset/train", transform=train_transform)
trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = datasets.ImageFolder("./dataset/test", transform=test_transform)
testloader = DataLoader(test_set, shuffle=False)

In [4]:
feature, target = next(iter(trainloader))
feature.shape

torch.Size([1, 3, 224, 224])

In [5]:
import torchvision
device = 'cuda'
model = torchvision.models.vgg11(pretrained=True).to(device)
for param in model.parameters():
    param.requires_grad = False

model.classifier._modules['6'] = nn.Linear(4096, 2)
model.to(device=device)
model

c:\Users\siddh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\siddh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [6]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss() # this is also called "criterion"/"cost function" in some places
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

In [7]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(trainloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.
    # scheduler.step()

    return last_loss

In [8]:
def accuracy(predictions, labels):
    classes = torch.argmax(predictions, dim=1)
    return torch.mean((classes == labels).float())

In [9]:
epoch_number = 0
from datetime import datetime
# torch.cuda.manual_seed(42)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
best_vloss = 1_000_000.
data_for_acc  =[]
EPOCHS = 20
for epoch in range(EPOCHS):
    running_accuracy = 0.00
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(testloader):
        vinputs, vlabels = vdata
        vinputs, vlabels = vinputs.to(device), vlabels.to(device)
        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)
        running_accuracy += accuracy(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    running_accuracy /= len(testloader)
    print('LOSS train {} test {} accuracy {}'.format(avg_loss, avg_vloss, running_accuracy))

    # Log the running loss averaged per batch
    # for both training and validation
  
    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
LOSS train 0.0 test 0.45014050602912903 accuracy 0.8181818127632141
EPOCH 2:
LOSS train 0.0 test 0.4260518252849579 accuracy 0.8429751992225647
EPOCH 3:
LOSS train 0.0 test 0.4090137779712677 accuracy 0.8429751992225647
EPOCH 4:
LOSS train 0.0 test 0.39822277426719666 accuracy 0.8512396216392517
EPOCH 5:
LOSS train 0.0 test 0.38907212018966675 accuracy 0.8347107172012329
EPOCH 6:
LOSS train 0.0 test 0.37823721766471863 accuracy 0.8429751992225647
EPOCH 7:
LOSS train 0.0 test 0.3720003366470337 accuracy 0.8347107172012329
EPOCH 8:
LOSS train 0.0 test 0.36572331190109253 accuracy 0.8347107172012329
EPOCH 9:
LOSS train 0.0 test 0.3605444133281708 accuracy 0.8429751992225647
EPOCH 10:
LOSS train 0.0 test 0.35700294375419617 accuracy 0.8512396216392517
EPOCH 11:
LOSS train 0.0 test 0.35539641976356506 accuracy 0.8595041036605835
EPOCH 12:
LOSS train 0.0 test 0.3518645167350769 accuracy 0.8677685856819153
EPOCH 13:
LOSS train 0.0 test 0.3518843352794647 accuracy 0.8595041036605835
E

In [10]:
torch.save(model, 'VGG.pth')

In [11]:
torch.cuda.empty_cache()